In [2]:
import pandas as pd

reviews = pd.read_csv("Data/winemag-data-130k-v2.csv", index_col=0)
reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


###  Grouping and Sorting


## 1. Groupwise Analysis

We can replicate what `.value_counts()` does by using the following:

In [4]:
reviews.groupby('points').points.count()

points
80       397
81       692
82      1836
83      3025
84      6480
85      9530
86     12600
87     16933
88     17207
89     12226
90     15410
91     11359
92      9613
93      6489
94      3758
95      1535
96       523
97       229
98        77
99        33
100       19
Name: points, dtype: int64

## How `.groupby()` Works in Pandas

### ✅ Concept:
`.groupby()` in pandas is used to **split** a DataFrame into groups based on the values in one or more columns, then optionally **apply** a function to each group (like `sum()`, `count()`, `mean()`, etc.).

---

### 🧠 Mechanism:
1. **Splitting**: 
   Pandas identifies all **unique values** in the column(s) you group by.
   
   Example:
   ```python
   reviews.groupby('points')
   ```
   This groups all rows by their `points` value.

2. **Internally**:
   - Creates a **GroupBy object**, which is a special object storing the mapping of labels → group of rows.
   - The data is not yet aggregated — it’s just logically split and ready for operations.

3. **Selecting a column**:
   ```python
   reviews.groupby('points').points
   ```
   This selects the `points` column from each group. It’s a grouped Series.

4. **Applying a function**:
   ```python
   reviews.groupby('points').points.count()
   ```
   This applies `.count()` to the `points` column in each group.
   Since every group consists of rows with the same point score, this counts **how many rows** are in each group.

---

### 🧪 Example:
Given:

```python
import pandas as pd

reviews = pd.DataFrame({
    'wine': ['A', 'B', 'C'],
    'points': [90, 91, 90],
    'price': [20, 25, 30]
})
```

Then:

```python
reviews.groupby('points').points.count()
```

**Output:**

```
points
90    2
91    1
Name: points, dtype: int64
```

This means:
- 2 wines have 90 points
- 1 wine has 91 points

---

### 🛠 Useful Variants:

- Count number of rows per group:
  ```python
  reviews.groupby('points').size()
  ```

- Get mean price per point score:
  ```python
  reviews.groupby('points').price.mean()
  ```

- Multiple aggregations:
  ```python
  reviews.groupby('points').agg({
      'price': ['mean', 'max'],
      'wine': 'count'
  })
  ```

- Reset index for easier access:
  ```python
  reviews.groupby('points').price.mean().reset_index()
  ```

---

### 🔁 Summary:
- `groupby()` groups data by column(s)
- It returns a lazy `GroupBy` object
- You must follow it with:
  - A column selection (e.g. `.points`)
  - And an aggregation (e.g. `.mean()`, `.count()`, etc.)

```python
# Full example
reviews.groupby('points').price.mean()
```

This gives the average wine price for each `points` score.


In [9]:
reviews.groupby('winery').apply(lambda df: df.title.iloc[0])

/var/folders/2p/d_nj_y2s0p32_2p156bn343c0000gn/T/ipykernel_12213/506966275.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reviews.groupby('winery').apply(lambda df: df.title.iloc[0])


winery
1+1=3                                     1+1=3 NV Rosé Sparkling (Cava)
10 Knots                            10 Knots 2010 Viognier (Paso Robles)
100 Percent Wine              100 Percent Wine 2015 Moscato (California)
1000 Stories           1000 Stories 2013 Bourbon Barrel Aged Zinfande...
1070 Green                  1070 Green 2011 Sauvignon Blanc (Rutherford)
                                             ...                        
Órale                       Órale 2011 Cabronita Red (Santa Ynez Valley)
Öko                    Öko 2013 Made With Organically Grown Grapes Ma...
Ökonomierat Rebholz    Ökonomierat Rebholz 2007 Von Rotliegenden Spät...
àMaurice               àMaurice 2013 Fred Estate Syrah (Walla Walla V...
Štoka                                    Štoka 2009 Izbrani Teran (Kras)
Length: 16757, dtype: object

The code above basically grouped by different wineries, took the first data of each group and returned only the title of that data back.

But grouping by can happen if you group by more than one thing. For example, the code below basically pick out the best wine by country and province.

So it basically groups by country, then by province. For each province, it takes the one that has the maximum points in that subgroup.


By the way, `idxmax()` returns the index value for the point with the highest value.

In [10]:
reviews.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.idxmax()])

/var/folders/2p/d_nj_y2s0p32_2p156bn343c0000gn/T/ipykernel_12213/1865732994.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reviews.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.idxmax()])


country  \
country   province                      
Argentina Mendoza Province  Argentina   
          Other             Argentina   
Armenia   Armenia             Armenia   
Australia Australia Other   Australia   
          New South Wales   Australia   
...                               ...   
Uruguay   Juanico             Uruguay   
          Montevideo          Uruguay   
          Progreso            Uruguay   
          San Jose            Uruguay   
          Uruguay             Uruguay   

                                                                  description  \
country   province                                                              
Argentina Mendoza Province  If the color doesn't tell the full story, the ...   
          Other             Take note, this could be the best wine Colomé ...   
Armenia   Armenia           Deep salmon in color, this wine offers a bouqu...   
Australia Australia Other   Writes the book on how to make a wine filled w...   
          New South Wales   De Bortoli's Noble One is as good as ever in 2...   
...                                                                       ...   
Uruguay   Juanico           This mature Bordeaux-style blend is earthy on ...   
          Montevideo        A rich, heady bouquet offers aromas of blackbe...   
          Progreso          Rusty in color but deep and complex in nature,...   
          San Jose          Baked, sweet, heavy aromas turn earthy with ti...   
          Uruguay           Cherry and berry aromas are ripe, healthy and ...   

                                                        designation  points  \
country   province                                                            
Argentina Mendoza Province                         Nicasia Vineyard      97   
          Other                                             Reserva      95   
Armenia   Armenia                                    Estate Bottled      88   
Australia Australia Other                             Sarah's Blend      93   
          New South Wales                        Noble One Bortytis      94   
...                                                             ...     ...   
Uruguay   Juanico                  Preludio Barrel Select Lote N 77      90   
          Montevideo        Monte Vide Eu Tannat-Merlot-Tempranillo      91   
          Progreso                   Etxe Oneko Fortified Sweet Red      90   
          San Jose                         El Preciado Gran Reserva      87   
          Uruguay                         Blend 002 Limited Edition      91   

                            price          province                 region_1  \
country   province                                                             
Argentina Mendoza Province  120.0  Mendoza Province                  Mendoza   
          Other              90.0             Other                    Salta   
Armenia   Armenia            15.0           Armenia                      NaN   
Australia Australia Other    15.0   Australia Other  South Eastern Australia   
          New South Wales    32.0   New South Wales          New South Wales   
...                           ...               ...                      ...   
Uruguay   Juanico            45.0           Juanico                      NaN   
          Montevideo         60.0        Montevideo                      NaN   
          Progreso           46.0          Progreso                      NaN   
          San Jose           50.0          San Jose                      NaN   
          Uruguay            22.0           Uruguay                      NaN   

                           region_2        taster_name taster_twitter_handle  \
country   province                                                             
Argentina Mendoza Province      NaN  Michael Schachner           @wineschach   
          Other                 NaN  Michael Schachner           @wineschach   
Armenia   Armenia               NaN      Mike DeSimon

`agg()` is also a function that you can use after `groupby()` that basically runs a statistic breakdown. Note that whatever you put inside `agg()` becomes the result's columns. Look at the example below.

As you can see, `agg()` takes a list of *names* of the functions you want to use.

In [12]:
reviews.groupby('country').price.agg([len, min, max]).head(10)

/var/folders/2p/d_nj_y2s0p32_2p156bn343c0000gn/T/ipykernel_12213/3169618422.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  reviews.groupby('country').price.agg([len, min, max]).head(10)
/var/folders/2p/d_nj_y2s0p32_2p156bn343c0000gn/T/ipykernel_12213/3169618422.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  reviews.groupby('country').price.agg([len, min, max]).head(10)


,len,min,max
country,,,
Argentina,3800,4.0,230.0
Armenia,2,14.0,15.0
Australia,2329,5.0,850.0
Austria,3345,7.0,1100.0
Bosnia and Herzegovina,2,12.0,13.0
Brazil,52,10.0,60.0
Bulgaria,141,8.0,100.0
Canada,257,12.0,120.0
Chile,4472,5.0,400.0


## 2. Multi-indexes

You can have multiple indexes stacked on top of each other in layers, instead of just one single-level index.

By the way, `len` function in agg basically counts all non-null values.

For example:

In [ ]:
reviews.groupby(['country', 'province']).description.agg([len]) # len basically counts non-null values 

len
country   province              
Argentina Mendoza Province  3264
          Other              536
Armenia   Armenia              2
Australia Australia Other    245
          New South Wales     85
...                          ...
Uruguay   Juanico             12
          Montevideo          11
          Progreso            11
          San Jose             3
          Uruguay             24

[425 rows x 1 columns]

Multi-indexing basically have their own thing which is a seperate whole thing like single-level indexes do. Though, we will talk about one function for multi-indexes which is `reset_index()`

In [17]:
reviews.groupby(['country', 'province']).description.agg([len]).reset_index()

,country,province,len
0,Argentina,Mendoza Province,3264
1,Argentina,Other,536
2,Armenia,Armenia,2
3,Australia,Australia Other,245
4,Australia,New South Wales,85
...,...,...,...
420,Uruguay,Juanico,12
421,Uruguay,Montevideo,11
422,Uruguay,Progreso,11
423,Uruguay,San Jose,3


## 3. Sorting

